In [1]:
# import libraries
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
# Assign sqlite database to db variable
db = sqlite3.connect('../data/hop_teaming.sqlite')

In [3]:
# Check the tables in the database
tables = db.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in tables:
    print(table[0])

cbsa
specialty
referrals
nppes
nashville_referrals


In [4]:
col = db.execute("PRAGMA table_info(referrals)")
print(col.fetchall())

[(0, 'index', 'INTEGER', 0, None, 0), (1, 'from_npi', 'INTEGER', 0, None, 0), (2, 'to_npi', 'INTEGER', 0, None, 0), (3, 'patient_count', 'INTEGER', 0, None, 0), (4, 'transaction_count', 'INTEGER', 0, None, 0), (5, 'average_day_wait', 'REAL', 0, None, 0), (6, 'std_day_wait', 'REAL', 0, None, 0)]


In [10]:
query = """
WITH nashville_providers AS (
    SELECT n.*

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
)

SELECT COUNT(*) AS tot_rows
    , COUNT(DISTINCT n.npi) AS unique_npis
    , COUNT(DISTINCT(CASE WHEN s.npi IS NOT NULL THEN s.npi ELSE NULL END)) AS npis_with_specialty
    , COUNT(DISTINCT(CASE WHEN rto.to_npi IS NOT NULL THEN rto.to_npi ELSE NULL END)) AS npis_who_refer_out
    , COUNT(DISTINCT(CASE WHEN rfrom.to_npi IS NOT NULL THEN rfrom.to_npi ELSE NULL END)) AS npis_who_get_referrals
    
FROM nashville_providers AS n

LEFT JOIN specialty AS s
    ON s.npi = n.npi
    
LEFT JOIN nashville_referrals AS rto
    ON rto.to_npi = n.npi
    
LEFT JOIN nashville_referrals AS rfrom
    ON rfrom.from_npi = n.npi
"""

In [11]:
nashville_providers = pd.read_sql(query, db)

In [12]:
nashville_providers

,tot_rows,unique_npis,npis_with_specialty,npis_who_refer_out,npis_who_get_referrals
0,179903132,38414,38414,8341,37867
